In [ ]:
import numpy as np
import scipy.stats as stats
from hansolo import *

Nb = 100
expert = "PWA"  # "EWA" or "PWA" or "Multilin"
learning = "blocs" # "blocs": objects presented by random blocs, or "random": each object is randomly chosen with replacement


m = 2501
W_output=np.load(f"{expert}_{learning}_W_HAN_ablation.npy")[:,:,m,:,:]


M_obj = 500 #size of testing set

dt = 0.002
N = 1000
T = N * dt

freq = 100  # firing rate input neurons +
freq2 = 150

alpha = np.array([100 * dt, 0])  # spontaneous rates of output neurons

p = freq * dt  # spiking proba input neurons
p2 = freq2 * dt

K_input = 12  # nb experts
n_input = 6  # nb input neurons
K_output = 2  # nb output neurons
N_a = n_input
n_feat = 6

obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
]
# blue circle blue square blue triangle red circle red square red triangle gray circle gray square gray triangle

answers = np.zeros((Nb, M_obj, N_a))  # 0 if wrong, 1 if correct


for n_a in range(N_a):
    K_input_new = K_input - 2*n_a
    n_input_new = n_input - n_a

    for nb in range(Nb):
        #creation of the list of objects (testing set)
        L=list_objects_random(obj,M_obj)
        feat2 = np.where(W_output[0,:,nb,n_a] != -1)[0]
        feat = feat2[:n_input_new]

        F_input = np.zeros((n_input_new, M_obj))
        F_output=np.zeros((K_output, M_obj))

    
        for m_obj in range(M_obj):

            obj_cur = L[m_obj]
            activity_cur = obj_cur[feat]
            cur_act = np.where(activity_cur == 1)[0]
            input_neurons=np.zeros((n_input_new, N))
            input_neurons[cur_act,:] = Poisson(freq, T, (cur_act.size, N))

            output_neurons=np.zeros((K_output, N))
            
                # simulation of the output neurons
            
            for j in range(K_output):
                probas = alpha[j] + np.sum(
                    (
                        W_output[j, feat, nb, n_a][:, np.newaxis]
                        - W_output[j, feat+n_feat, nb, n_a][:, np.newaxis]
                    )
                    * input_neurons[:,:],
                    axis=0,
                )
                clipped_probas = np.clip(probas, 0, 1)
                output_neurons[j, :] = stats.bernoulli.rvs(clipped_probas)

            # firing rates
            F_input[:, m_obj] = np.sum(input_neurons, axis=1) / T
            F_output[:, m_obj] = np.sum(output_neurons, axis=1) / T

            
            if (in_A(obj_cur) and F_output[0, m_obj] > F_output[1, m_obj]) or (
                in_B(obj_cur) and F_output[1, m_obj] > F_output[0, m_obj]
            ):
                answers[nb, m_obj, n_a] = 1
    print(n_a)
        


np.save(f"{expert}_{learning}_HAN_test_ablation",answers)
#np.save(f"{expert}_{learning}_W_HAN", W_output)
#np.save(f"{expert}100F_in", F_output)
